In [431]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np

from sklearn.ensemble import RandomForestClassifier

from math import sqrt

from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
#from keras.datasets import mnist

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler



In [432]:
currency_list = [#'USDCHF1440',
                 'GBPUSD1440',
    #'EURUSD1440', 
    #'USDJPY1440', 
    #'USDCAD1440', 
    #'AUDUSD1440', 
    #'NZDUSD1440',
                 #'GBPCHF1440',
    #'EURCHF1440', 
    #'CHFJPY1440', 
    #'CADCHF1440',
    #'AUDCHF1440', 
    #'NZDCHF1440', 
    #'EURGBP1440',
             #   'GBPCAD1440',
     #'GBPAUD1440', 
    #'EURJPY1440',
    #'EURCAD1440',
    #'EURAUD1440',
    #'EURNZD1440',
    #'CADJPY1440', 
    #'AUDJPY1440',
    #'NZDJPY1440',
    #'AUDCAD1440', 
    #'NZDCAD1440', 
                #'AUDNZD1440'
                ]



# for q in currency_list:
    
#     errors = []
    
#     for x in range(5):

#         currency = q.replace('1440','')

#         data = pd.read_excel('files/currency_training_data/' + currency +'_combine_data_dataframe.xlsx', sheet_name=0)
#         #data = data.head(695)


#         X = data.drop(columns=['Unnamed: 0', 
#                                'date_start',  'nextweek_class',


#                               ])



#         y = data['nextweek_class']

#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2 )



#         lnr= LinearRegression()
#         lnr.fit(X_train, y_train)
#         y_predict = lnr.predict(X_test)
        
        
#         error = sqrt(mean_squared_error(y_test, y_pred))
#         errors.append(error)
       
        
#     average_error = sum(errors)/len(errors)
       
#     print(q + " Linear regression Average " + str(average_error))
    
    
   


In [ ]:
for q in currency_list:
    
    
    
    sequences = [1,]
    
    all_sequence_result = []
        
    for m in sequences:
    
        errors = []
        for x in range(30):
        
            currency = q.replace('1440','')

            data = pd.read_excel('files/currency_training_data/' + q +'_combine_data_dataframe.xlsx', sheet_name=0)
            #data = data.head(695)


            X = data.drop(columns=['Unnamed: 0', 
                                   'date_start',  'nextweek_class',


                                  ])



            y = data['nextweek_class']


#             X_scaler = RobustScaler(quantile_range=(10.0, 90.0),)
            
#             X_scaler.fit(X)
           
#             X = X_scaler.transform(X)
           
#             X = pd.DataFrame(X, columns = [q+"_class", q+'_volume'])
            

            #print(X)


            # after scaling the df, resulted in "scaled_dataset"





            result = []
            # for loop will walk for each of the 1500 rows
            for i in range(0,len(X)):
                # every group must have the same length, so if current loop position i + number 
                # of sequences is higher than df length, breaks
                if i+m <= len(X):
                    # this will add into the list as [[R1a,R1b...R1t],[R2a,R2b...R2t],...[R5a,R5b...R5t]]
                    result.append(X[i:i+m].values)
            # Converting to array + keras takes float32 better than 64
            train_x = np.array(result)
            #train_x  = train_x.astype('float32')
            # making the y into same length as X
            train_y = np.array(y.head(len(train_x)).values)

            print(train_x.shape, train_y.shape)
            #print(train_x[len(train_x)-10])
            #print(train_y[len(train_x)-10])
            
            
           

            X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.15 )
                               
            

            #X_train, X_val, y_train, y_val = train_test_split( X_train, y_train, test_size = 0.15 )

            

            #Initializing the classifier Network
            classifier = Sequential()

            #Adding the input LSTM network layer
            #classifier.add(CuDNNLSTM(128, input_shape=(X_train.shape[1:]), return_sequences=True))
            classifier.add(LSTM(100, input_shape=(train_x.shape[1:]), return_sequences=True))
           
           
            classifier.add(LSTM(100,  return_sequences=True), )
            #classifier.add(LSTM(100,  return_sequences=True))
            

            #classifier.add(Dense(units = 1))
            classifier.add(LSTM(100,  return_sequences=False))
            #classifier.add(Dropout(0.2))
            #Adding a second LSTM network layer

            #classifier.add(LSTM(128))
            #Adding a dense hidden layer
            #classifier.add(Dense(64, activation='relu'))
            #classifier.add(Dropout(0.2))

            #Adding the output layer
            #classifier.add(Dense(35, activation='softmax'))

            #Compiling the network
            classifier.compile( loss='mean_squared_error',
                            optimizer=Adam(learning_rate=0.001, decay=1e-6),
                             )

            #print(classifier.summary())

            #Fitting the data to the model
            history = classifier.fit(X_train,
                        y_train,
                        epochs=5,
                        validation_data=(X_test, y_test)
                                    )        

            val_loss  = classifier.evaluate(X_test, y_test)
            error = sqrt(val_loss)
            errors.append(error)
            plt.plot(history.history['loss'],'red')
            plt.plot(history.history['val_loss'], 'blue')
            print('------------------------------------------------------------------------------------',x)
        average_error = sum(errors)/len(errors)
        print(errors)
        print(q , "------------------------ RNN " , average_error)
        all_sequence_result.append(str(m)+" sequence" )
        all_sequence_result.append(average_error)


(969, 1, 2) (969,)
Epoch 1/5
26/26 [==============================] - 12s 111ms/step - loss: 134.3039 - val_loss: 137.8347
Epoch 2/5
26/26 [==============================] - 1s 37ms/step - loss: 134.1902 - val_loss: 137.9800
Epoch 3/5
26/26 [==============================] - 1s 39ms/step - loss: 133.9090 - val_loss: 138.3350
Epoch 4/5
26/26 [==============================] - 1s 38ms/step - loss: 133.6292 - val_loss: 138.6577
Epoch 5/5
5/5 [==============================] - 0s 12ms/step - loss: 138.8952
------------------------------------------------------------------------------------ 0
(969, 1, 2) (969,)
Epoch 1/5
26/26 [==============================] - 11s 114ms/step - loss: 134.6408 - val_loss: 135.9736
Epoch 2/5
26/26 [==============================] - 1s 40ms/step - loss: 134.4997 - val_loss: 136.2151
Epoch 3/5
26/26 [==============================] - 1s 40ms/step - loss: 134.2323 - val_loss: 136.5261
Epoch 4/5
26/26 [==============================] - 1s 40ms/step - loss: 134.02

(969, 1, 2) (969,)
Epoch 1/5
26/26 [==============================] - 11s 110ms/step - loss: 133.7950 - val_loss: 140.8206
Epoch 2/5
26/26 [==============================] - 1s 38ms/step - loss: 133.7512 - val_loss: 140.7442
Epoch 3/5
26/26 [==============================] - 1s 39ms/step - loss: 133.6662 - val_loss: 140.5742
Epoch 4/5
26/26 [==============================] - 1s 39ms/step - loss: 133.5905 - val_loss: 140.4912
Epoch 5/5
5/5 [==============================] - 0s 14ms/step - loss: 140.3647
------------------------------------------------------------------------------------ 12
(969, 1, 2) (969,)
Epoch 1/5
26/26 [==============================] - 12s 108ms/step - loss: 132.3268 - val_loss: 149.1087
Epoch 2/5
26/26 [==============================] - 1s 36ms/step - loss: 132.3080 - val_loss: 149.0507
Epoch 3/5
26/26 [==============================] - 1s 36ms/step - loss: 132.2337 - val_loss: 149.0079
Epoch 4/5
26/26 [==============================] - 1s 37ms/step - loss: 132.1

(969, 1, 2) (969,)
Epoch 1/5
26/26 [==============================] - 11s 111ms/step - loss: 132.2657 - val_loss: 149.2925
Epoch 2/5
26/26 [==============================] - 1s 37ms/step - loss: 132.2041 - val_loss: 149.1805
Epoch 3/5
26/26 [==============================] - 1s 37ms/step - loss: 132.0851 - val_loss: 149.1162
Epoch 4/5
26/26 [==============================] - 1s 38ms/step - loss: 131.9733 - val_loss: 148.9885
Epoch 5/5
5/5 [==============================] - 0s 14ms/step - loss: 148.9253
------------------------------------------------------------------------------------ 24
(969, 1, 2) (969,)
Epoch 1/5
26/26 [==============================] - 12s 107ms/step - loss: 138.1483 - val_loss: 116.1996
Epoch 2/5
26/26 [==============================] - 1s 36ms/step - loss: 138.1027 - val_loss: 116.0951
Epoch 3/5
26/26 [==============================] - 1s 36ms/step - loss: 138.0075 - val_loss: 115.8345
Epoch 4/5
26/26 [==============================] - 1s 36ms/step - loss: 137.8

In [ ]:
print(all_sequence_result)   
